In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import chi2, mutual_info_classif, SelectKBest, f_classif
from sklearn.model_selection import cross_val_score 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, roc_curve, auc
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split

#Load data from csv file
data = pd.read_csv("datapbmc10xV2NormWithColNamesRowNames.csv")
data.head(10) 

data.set_index("Unnamed: 0", inplace = True)
data.index.name = None

X = data.iloc[:, data.columns != 'cellType'].values
#X=data.iloc[:,0:4915]
X = np.absolute(X)

y = data.iloc[:,data.columns == 'cellType'].values.ravel()

def precision_MC(y_true, y_pred):
    TP = np.diag(confusion_matrix(y_true, y_pred))
    FP = confusion_matrix(y_true, y_pred).sum(axis=1) - np.diag(confusion_matrix(y_true, y_pred))
    FP = FP.astype(float)
    TP = TP.astype(float)
    precision = TP/(TP+FP) * 100
    return precision

def recall_MC(y_true, y_pred):
    TP = np.diag(confusion_matrix(y_true, y_pred))
    FN = confusion_matrix(y_true, y_pred).sum(axis=0) - np.diag(confusion_matrix(y_true, y_pred))
    FN = FN.astype(float)
    TP = TP.astype(float)
    recall =  TP/(TP+FN) * 100
    return recall

def accuracy_MC(y_true, y_pred):
    FP = confusion_matrix(y_true, y_pred).sum(axis=1) - np.diag(confusion_matrix(y_true, y_pred))  
    FN = confusion_matrix(y_true, y_pred).sum(axis=0) - np.diag(confusion_matrix(y_true, y_pred))
    TP = np.diag(confusion_matrix(y_true, y_pred))
    TP = TP.astype(float)
    TN = confusion_matrix(y_true, y_pred).sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TN = TN.astype(float)
    accuracy = (TP+TN)/(TP+FP+FN+TN) * 100
    return accuracy

def npval_MC(y_true, y_pred): 
    FN = confusion_matrix(y_true, y_pred).sum(axis=0) - np.diag(confusion_matrix(y_true, y_pred))
    FN = FN.astype(float)
    TP = np.diag(confusion_matrix(y_true, y_pred))
    TP = TP.astype(float)
    FP = confusion_matrix(y_true, y_pred).sum(axis=1) - np.diag(confusion_matrix(y_true, y_pred)) 
    FP = FP.astype(float)
    TN = confusion_matrix(y_true, y_pred).sum() - (FP + FN + TP)
    TN = TN.astype(float)
    npv = TN/(TN+FN) * 100
    return npv

def specificity_MC(y_true, y_pred): 
    FP = confusion_matrix(y_true, y_pred).sum(axis=1) - np.diag(confusion_matrix(y_true, y_pred)) 
    FP = FP.astype(float)
    FN = confusion_matrix(y_true, y_pred).sum(axis=0) - np.diag(confusion_matrix(y_true, y_pred))
    FN = FN.astype(float)
    TP = np.diag(confusion_matrix(y_true, y_pred))
    TP = TP.astype(float)
    TN = confusion_matrix(y_true, y_pred).sum() - (FP + FN + TP)
    TN = TN.astype(float)
    spec = TN/(TN+FP)  * 100
    return spec


def plot_ROC_curve_MC(y_true, y_pred, FSName):
    n_classes = len(np.unique(y_true))
    y_true = label_binarize(y_true, classes=np.arange(n_classes))
    #y_pred = label_binarize(y_pred, classes=np.arange(n_classes))
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes
    
    # Plot all ROC curves
    plt.figure()
    #colors = cycle(["aqua", "darkorange", "cornflowerblue"])
    for i in range(n_classes):
        plt.plot(
            fpr[i],
            tpr[i],
            label="ROC curve of class {0} (area = {1:0.4f})".format(i, roc_auc[i]),
        )

    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve of " + str(FSName))
    plt.legend(loc="lower right")
    path = "BIBM_Workshop/Results_RF_Bayesian/ROC_"+ str(FSName)
    plt.savefig(f'{path}.png')

#Bayesian Search
bayesian_search = BayesSearchCV(
        RandomForestClassifier(),
        {
            'criterion': Categorical(['gini','entropy']),
            'max_depth': Integer(1,60),
            'n_estimators':(100,500),
            'max_features': (['sqrt','log2'])
            #'max_leaf_nodes':Integer(0,prior='log-uniform'),
            #'min_samples_leaf':Real(0.1,0.5,prior='uniform'),
            #'min_samples_split':Real(0.1,1,prior='uniform')
        },
        n_iter=50,scoring='accuracy',n_jobs = -1, verbose = 10,
        random_state=0, cv=10
    )

#grid_search = GridSearchCV(SVC(), tuned_params, cv=10, scoring='accuracy',n_jobs = -1, verbose = 10)
bayesian_search.fit(X, y)
bayesian_search.best_params_
print("")
print("Best Parameters Found")
print(bayesian_search.best_params_) 
clf = RandomForestClassifier(criterion = bayesian_search.best_params_['criterion'], max_depth = bayesian_search.best_params_['max_depth'], n_estimators= bayesian_search.best_params_['n_estimators'],
      max_features= bayesian_search.best_params_['max_features'])

#clf = SVC(kernel = 'linear', C= 0.1)
k_range = [100,200,300,400]
acc_chi2 = []
acc_mi = []
acc_annova = []

for i in k_range:    
    X_kbest_features_annova = SelectKBest(f_classif, k=i).fit_transform(X, y)
    
    acc = cross_val_score(clf, X_kbest_features_annova, y, cv=10, scoring="accuracy").mean() 
    acc_annova.append(acc*100)
    print(acc_annova)
    
    X_kbest_features_chi2 = SelectKBest(chi2, k=i).fit_transform(X, y)
    
    acc1 = cross_val_score(clf, X_kbest_features_chi2, y, cv=10, scoring="accuracy").mean() 
    acc_chi2.append(acc1*100)
    print(acc_chi2)
    
    X_kbest_features_mi = SelectKBest(mutual_info_classif, k=i).fit_transform(X,y)
    
    acc2 = cross_val_score(clf, X_kbest_features_mi, y, cv=10, scoring="accuracy").mean()
    acc_mi.append(acc2*100)
    print(acc_mi)

print("Accuracy Anova", acc_annova)
print("Chi2 accuracy", acc_chi2)
print("IG accuracy", acc_mi)

optimal_k_annova = k_range[acc_annova.index(max(acc_annova))]
optimal_k_chi2 = k_range[acc_chi2.index(max(acc_chi2))]
optimal_k_mi = k_range[acc_mi.index(max(acc_mi))]

# multiple line plot of all Feature selection methods
plt.plot( k_range, acc_annova,color='blue', linewidth=2, label = "Anova F-value")
plt.plot( k_range, acc_chi2, color='orange', linewidth=2, label ="Chi2")
plt.plot( k_range, acc_mi, color='green', linewidth=2, label="Information Gain")
plt.legend()
plt.xlabel("Number of Features k")
plt.ylabel("Accuracy")
plt.show()
plt.savefig('BIBM_Workshop/Results_RF_Bayesian/Bestk.png')

data1 = data
data1.drop(columns='cellType', inplace = True)
data1.head(10)

df_columns = pd.DataFrame(data1.columns)

#Printing top 20 features_anova
bestfeatures_anova = SelectKBest(f_classif, k= optimal_k_annova)
fit_anova = bestfeatures_anova.fit(X,y)
df_scores_anova = pd.DataFrame(fit_anova.scores_)

# concatenate dataframes
feature_scores_anova = pd.concat([df_columns, df_scores_anova],axis=1)
feature_scores_anova.columns = ['Feature_Name','Score']  # name output columns
print("Anova")
print(feature_scores_anova.nlargest(20,'Score')) 
# export selected features to .csv
df_feat_anova = feature_scores_anova.nlargest(20,'Score')
df_feat_anova.to_csv('/home/horas/sheena/BIBM_Workshop/Results_RF_Bayesian/feature_selected_Anova.csv', index=False)

#Printing top 20 features_chi2
bestfeatures_chi2 = SelectKBest(chi2, k= optimal_k_chi2)
fit_chi2 = bestfeatures_chi2.fit(X,y)
df_scores_chi2 = pd.DataFrame(fit_chi2.scores_)

# concatenate dataframes
feature_scores_chi2 = pd.concat([df_columns, df_scores_chi2],axis=1)
feature_scores_chi2.columns = ['Feature_Name','Score']  # name output columns
print("CHI2")
print(feature_scores_chi2.nlargest(20,'Score')) 
# export selected features to .csv
df_feat_chi2 = feature_scores_chi2.nlargest(20,'Score')
df_feat_chi2.to_csv('BIBM_Workshop/Results_RF_Bayesian/feature_selected_Chi2.csv', index=False)

#Printing top 20 features_IG
bestfeatures_ig = SelectKBest(mutual_info_classif, k= optimal_k_mi)
fit_ig = bestfeatures_ig.fit(X,y)
df_scores_ig = pd.DataFrame(fit_ig.scores_)

# concatenate dataframes
feature_scores_ig = pd.concat([df_columns, df_scores_ig],axis=1)
feature_scores_ig.columns = ['Feature_Name','Score']  # name output columns
print("IG")
print(feature_scores_ig.nlargest(20,'Score')) 
# export selected features to .csv
df_feat_ig = feature_scores_ig.nlargest(20,'Score')
df_feat_ig.to_csv('BIBM_Workshop/Results_RF_Bayesian/feature_selected_IG.csv', index=False)

perf_cols = ["Cell Type", "Accuracy%","Precision%","Recall%","Negative Predictive Value%","Specificity%","F1-score%"]
perf_anova  = pd.DataFrame(columns = perf_cols)
perf_chi2  = pd.DataFrame(columns = perf_cols)
perf_mi  = pd.DataFrame(columns = perf_cols)

label = np.unique(y.tolist())
class_lbl = [int(i) for i in label] 
class_lbl = ['CellType ' + str(s) for s in class_lbl]

#Feature selection
X_kbestfeatures_annova = SelectKBest(f_classif, k = optimal_k_annova).fit_transform(X, y)
X_kbestfeatures_chi2 = SelectKBest(chi2, k = optimal_k_chi2).fit_transform(X, y)
X_kbestfeatures_mi = SelectKBest(mutual_info_classif, k = optimal_k_mi).fit_transform(X, y)

print(X_kbestfeatures_annova.shape) #(1724, 300)
print(X_kbestfeatures_chi2.shape) #(1724, 400)
print(X_kbestfeatures_mi.shape) #(1724, 200)

#Anova + classification Method
random_state = np.random.RandomState(0)

X_train_anova, X_test_anova, y_train_anova, y_test_anova = train_test_split(X_kbestfeatures_annova, y, test_size = 0.30, stratify=y, random_state= random_state)
clf.fit(X_train_anova, y_train_anova)
y_pred_anova = clf.predict(X_test_anova)

X_train_anova.shape #(1351,400) , (1206, 300)
X_test_anova.shape #(579, 400), (518, 300)

anova_accuracy = accuracy_score(y_test_anova, y_pred_anova)
print("Anova Accuracy")
print(anova_accuracy) #0.9792746113989638, 0.8687258687258688

pre_anova = precision_MC(y_test_anova,y_pred_anova)
rec_anova = recall_MC(y_test_anova,y_pred_anova)
acc_anova = accuracy_MC(y_test_anova,y_pred_anova)
npv_anova = npval_MC(y_test_anova,y_pred_anova)
spe_anova = specificity_MC(y_test_anova,y_pred_anova)
f1_anova = 2* (pre_anova * rec_anova)/(pre_anova + rec_anova)

print("RF MultiClass and Anova")
print(pre_anova)
print(rec_anova)
print(acc_anova)
print(npv_anova)
print(spe_anova)
print(f1_anova)

perf_anova['Cell Type'] = class_lbl
perf_anova['Precision%'] = pre_anova.tolist()
perf_anova['Accuracy%'] =  acc_anova.tolist()
perf_anova['Recall%']  = rec_anova.tolist()
perf_anova['Negative Predictive Value%'] = npv_anova.tolist()
perf_anova['Specificity%'] = spe_anova.tolist()
perf_anova['F1-score%'] = f1_anova.tolist()

perf_anova.to_csv('BIBM_Workshop/Results_RF_Bayesian/PerformanceMetrics_Anova.csv', index = False)

#Plot ROC curve for OneVsRest + Anova
#y_score_anova = classifier.fit(X_train_anova, y_train_anova).decision_function(X_test_anova)

n_classes = len(np.unique(y_test_anova))
y_pred_an = label_binarize(y_pred_anova, classes=np.arange(n_classes))

plot_ROC_curve_MC(y_test_anova, y_pred_an, "Anova")

#Chi2 + classification Method
X_train_chi2, X_test_chi2, y_train_chi2, y_test_chi2 = train_test_split(X_kbestfeatures_chi2, y, test_size = 0.30, stratify=y, random_state= random_state)
clf.fit(X_train_chi2, y_train_chi2)
y_pred_chi2 = clf.predict(X_test_chi2)

X_train_anova.shape #(1351,400), (1206, 400)
X_test_anova.shape #(579, 400), (518, 400)

chi2_accuracy = accuracy_score(y_test_chi2, y_pred_chi2)
print("Chi2 Accuracy")
print(chi2_accuracy) #0.9585492227979274, 0.6621621621621622

pre_chi2 = precision_MC(y_test_chi2,y_pred_chi2)
rec_chi2 = recall_MC(y_test_chi2,y_pred_chi2)
acc_chi2 = accuracy_MC(y_test_chi2,y_pred_chi2)
npv_chi2 = npval_MC(y_test_chi2,y_pred_chi2)
spe_chi2 = specificity_MC(y_test_chi2,y_pred_chi2)
f1_chi2 = 2* (pre_chi2 * rec_chi2)/(pre_chi2 + rec_chi2)

print("SVC MultiClass and Chi2")
print(pre_chi2)
print(rec_chi2)
print(acc_chi2)
print(npv_chi2)
print(spe_chi2)
print(f1_chi2)

perf_chi2['Cell Type'] = class_lbl
perf_chi2['Precision%'] = pre_chi2.tolist()
perf_chi2['Accuracy%'] =  acc_chi2.tolist()
perf_chi2['Recall%']  = rec_chi2.tolist()
perf_chi2['Negative Predictive Value%'] = npv_chi2.tolist()
perf_chi2['Specificity%'] = spe_chi2.tolist()
perf_chi2['F1-score%'] = f1_chi2.tolist()

perf_chi2.to_csv('BIBM_Workshop/Results_RF_Bayesian/PerformanceMetrics_Chi2.csv', index = False)

#Plot ROC curve for OneVsRest + Chi2
#y_score_chi2 = classifier.fit(X_train_chi2, y_train_chi2).decision_function(X_test_chi2)

n_classes = len(np.unique(y_test_chi2))
y_pred_chi = label_binarize(y_pred_chi2, classes=np.arange(n_classes))

plot_ROC_curve_MC(y_test_chi2, y_pred_chi, "Chi2")

#IG + classification Method
X_train_mi, X_test_mi, y_train_mi, y_test_mi = train_test_split(X_kbestfeatures_mi, y, test_size = 0.30, stratify=y, random_state= random_state)
clf.fit(X_train_mi, y_train_mi)
y_pred_mi = clf.predict(X_test_mi)

X_train_mi.shape #(1351,400), (1206, 400)
X_test_mi.shape #(579, 400), (518, 400)

mi_accuracy = accuracy_score(y_test_mi, y_pred_mi)
print("IG Accuracy")
print(mi_accuracy) #0.9775474956822107, 0.972972972972973

pre_mi = precision_MC(y_test_mi,y_pred_mi)
rec_mi = recall_MC(y_test_mi,y_pred_mi)
acc_mi = accuracy_MC(y_test_mi,y_pred_mi)
npv_mi = npval_MC(y_test_mi,y_pred_mi)
spe_mi = specificity_MC(y_test_mi,y_pred_mi)
f1_mi = 2* (pre_mi * rec_mi)/(pre_mi + rec_mi)

print("SVC MultiClass and IG")
print(pre_mi)
print(rec_mi)
print(acc_mi)
print(npv_mi)
print(spe_mi)
print(f1_mi)

perf_mi['Cell Type'] = class_lbl
perf_mi['Precision%'] = pre_mi.tolist()
perf_mi['Accuracy%'] =  acc_mi.tolist()
perf_mi['Recall%']  = rec_mi.tolist()
perf_mi['Negative Predictive Value%'] = npv_mi.tolist()
perf_mi['Specificity%'] = spe_mi.tolist()
perf_mi['F1-score%'] = f1_mi.tolist()

perf_mi.to_csv('BIBM_Workshop/Results_RF_Bayesian/PerformanceMetrics_IG.csv', index = False)

#Plot ROC curve for OneVsRest + IG
#y_score_mi = classifier.fit(X_train_mi, y_train_mi).decision_function(X_test_mi)

n_classes = len(np.unique(y_test_mi))
y_pred_ig = label_binarize(y_pred_mi, classes=np.arange(n_classes))

plot_ROC_curve_MC(y_test_mi, y_pred_ig, "IG")

## Tabular Representation of Efficiency
table11= plt.table(cellText=perf_anova.values,
          colLabels=perf_anova.columns,
          cellLoc = 'center', rowLoc = 'center', colWidths=[0.25 for x in perf_anova.columns],
          loc='center')
table11.auto_set_font_size(False)
table11.set_fontsize(10)
table11.scale(2, 2)
plt.title('Results - RF with Anova', pad = '80.0')
plt.axis('off')

## Tabular Representation of Efficiency
table22= plt.table(cellText=perf_chi2.values,
          colLabels=perf_chi2.columns,
          cellLoc = 'center', rowLoc = 'center', colWidths=[0.25 for x in perf_chi2.columns],
          loc='center')
table22.auto_set_font_size(False)
table22.set_fontsize(10)
table22.scale(2, 2)
plt.title('Results - RF with Chi2', pad = '80.0')
plt.axis('off')

## Tabular Representation of Efficiency
table33= plt.table(cellText=perf_mi.values,
          colLabels=perf_mi.columns,
          cellLoc = 'center', rowLoc = 'center', colWidths=[0.25 for x in perf_mi.columns],
          loc='center')
table33.auto_set_font_size(False)
table33.set_fontsize(10)
table33.scale(2, 2)
plt.title('Results - RF with Information Gain', pad = '80.0')
plt.axis('off')



